# Create the Topography of the 79NG Fjord GETM Setup

This notebook creates the topography of the 79° North Glacier (79NG) fjord in a format that is suitable for GETM.
The topography consists of the bathymetry, extracted from the RTopo dataset, and the ice thickness, extracted from the BedMachine dataset.
Furthermore, the indices of the grid points that form the grounding line are determined.

To use this notebook:
 1. Download the file `RTopo-2.0.4_30sec_Greenland.nc` from the [dataset by Schaffer et al. (2019)](https://doi.org/10.1594/PANGAEA.905295) into a folder `data`.
 2. Download the file `BedMachineGreenland-v5.nc` from the [dataset by Morlighem et al. (2022)](https://doi.org/10.5067/GMEVBWFLWA7X) into a folder `data`.

Currently, a resolution of about 1 km ($\Delta \phi = 1/120° \approx 0.0083°$, $\Delta \lambda = 6/120° = 0.05°$) is implemented.
A resolution of about 500 m ($\Delta \phi = 1/240° \approx 0.00417°$, $\Delta \lambda = 3/120° = 0.025°$) is planned.

**Acknowledgement:**
Many thanks to Samira Zander for valuable preparatory work during an internship at IOW in 2021.

**References:**
 * Morlighem, M.  et al. (2022): _IceBridge BedMachine Greenland, Version 5,_ NASA National Snow and Ice Data Center Distributed Active Archive Center, https://doi.org/10.5067/GMEVBWFLWA7X (dataset)
 * Reinert, M., et al. (2023): _High-Resolution Simulations of the Plume Dynamics in an Idealized 79°N Glacier Cavity Using Adaptive Vertical Coordinates,_ Journal of Advances in Modeling Earth Systems, https://doi.org/10.1029/2023MS003721
 * Schaffer, J. et al. (2019): _An update to Greenland and Antarctic ice sheet topography, cavity geometry, and global bathymetry (RTopo-2.0.4),_ PANGAEA, https://doi.org/10.1594/PANGAEA.905295 (dataset)

Notebook by Markus Reinert (IOW, 2023–2024, https://orcid.org/0000-0002-3761-8029).

In [ ]:
import gsw
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cmocean
from pyproj import CRS, Transformer

from tools.smoothing import smooth_2D_array
from tools.configuration import Configuration

In [ ]:
config = Configuration()

## Prepare the coordinate transformation

### CRS of RTopo

The RTopo dataset is in lat–lon coordinates, which we also want to use for the GETM setup.
RTopo has a resolution of $\Delta \phi = \Delta \lambda = 1/120°$, which is highely anisotropic at 79°N, i.e., the distance in meters is much larger in meridional than in longitudinal direction: $\Delta y \gg \Delta x$.
To obtain an approximately squared model grid ($\Delta x \approx \Delta y$), we have to choose a coarser resolution in longitudinal than in meridional direction: $\Delta \lambda > \Delta \phi$.
The anisotropy also has the effect that lat–lon coordinates are not feasible for interpolation.

In [ ]:
crs_latlon = CRS.from_epsg(4326)
crs_latlon

### CRS of BedMachine

The BedMachine dataset is in Cartesian coordinates.
Since distances in this coordinate system are close to physical distances on Earth, we can use these coordinates for interpolation.

In [ ]:
crs_cartesian = CRS.from_epsg(3413)
crs_cartesian

### CRS transformation

In [ ]:
transform_latlon_to_xy = Transformer.from_crs(crs_latlon, crs_cartesian).transform

## Load the RTopo dataset

In [ ]:
rtopo = xr.open_dataset("data/RTopo-2.0.4_30sec_Greenland.nc")

# Correct the names of the dimensions ("lat"/"lon" instead of "latdim"/"londim")
rtopo = rtopo.set_index(latdim="lat", londim="lon")
rtopo = rtopo.rename({"latdim": "lat", "londim": "lon"})

# Choose an area around 79NG that is a bit larger than the desired model domain
rtopo = rtopo.sel(lon=slice(-23, -14), lat=slice(79.1, 80.4))

# Check that the mask of the floating ice tongue (amask = 2) is simply given by all
# points at which the ice draft is below the sea level but above the sea floor.
# This will allow us later to re-construct the mask from regridded data.
assert np.all(
    ((rtopo.ice_base_topography > rtopo.bedrock_topography) & (rtopo.ice_base_topography < 0))
    == (rtopo.amask == 2)
), "mask of floating ice tongue cannot be reconstructed as expected"

# Compute and add Cartesian coordinates
X, Y = transform_latlon_to_xy(*xr.broadcast(rtopo.lat, rtopo.lon))
rtopo.coords["x"] = (["lat", "lon"], X)
rtopo.coords["y"] = (["lat", "lon"], Y)
rtopo.x.attrs = {"long_name": crs_cartesian.axis_info[0].name, "units": "m", "CRS": str(crs_cartesian)}
rtopo.y.attrs = {"long_name": crs_cartesian.axis_info[1].name, "units": "m", "CRS": str(crs_cartesian)}

rtopo

### Compute the grid resolution

In [ ]:
# Compute grid resolution in degrees
d_lat = np.diff(rtopo.lat)
d_lon = np.diff(rtopo.lon)

# Compute grid resolution in meters
LON, LAT = np.meshgrid(rtopo.lon, rtopo.lat)
dist_lat = gsw.distance(LON, LAT, axis=0)
dist_lon = gsw.distance(LON, LAT, axis=1)

print(f"Meridional   resolution: {d_lat.min():.6f}° to {d_lat.max():.6f}° ({dist_lat.min():.2f} m to {dist_lat.max():.2f} m)")
print(f"Longitudinal resolution: {d_lon.min():.6f}° to {d_lon.max():.6f}° ({dist_lon.min():.2f} m to {dist_lon.max():.2f} m)")

### Show the data

In [ ]:
fig, axs = plt.subplots(ncols=3, sharex=True, sharey=True, constrained_layout=True, dpi=300, figsize=(10, 3))
fig2, axs2 = plt.subplots(ncols=3, sharex=True, sharey=True, constrained_layout=True, dpi=300, figsize=(10, 2.4))

fig.suptitle(f"Ellipsoidal coordinates ({crs_latlon})")
fig2.suptitle(f"Cartesian coordinates ({crs_cartesian})")

for ax, ax2, var, cmap in zip(
    axs, axs2, ["bedrock_topography", "ice_base_topography", "amask"], [cmocean.cm.topo, None, None]
):
    im = rtopo[var].plot(ax=ax, cmap=cmap)
    im2 = rtopo[var].plot(ax=ax2, x="x", y="y", cmap=cmap)
    ax.set_title(var)
    ax2.set_title(var)
    ax2.set_aspect("equal")
    if var == "amask":
        im.colorbar.set_ticks(np.arange(4))
        im2.colorbar.set_ticks(np.arange(4))
for ax in [*axs[1:], *axs2[1:]]:
    ax.set_ylabel("")

## Load the BedMachine dataset

In [ ]:
bedmachine = xr.open_dataset("data/BedMachineGreenland-v5.nc")
# Check that the expected coordinate reference system is used
assert str(crs_cartesian).lower() in bedmachine.proj4, f"BedMachine seems to use a coordinate system other than {crs_cartesian}"
# Invert the y-axis to have increasing coordinates
bedmachine = bedmachine.isel(y=slice(None, None, -1))
# Crop to the same extent as RTopo
bedmachine = bedmachine.sel(x=slice(rtopo.x.min(), rtopo.x.max()), y=slice(rtopo.y.min(), rtopo.y.max()))

bedmachine

### Compute the grid resolution

In [ ]:
dx = np.diff(bedmachine.x)
dy = np.diff(bedmachine.y)

assert all(dx == dx[0]), "x-resolution is not constant"
assert all(dy == dy[0]), "y-resolution is not constant"
assert dx[0] == dy[0], "x- and y-resolutions differ"

print(f"Resolution in x- and in y-direction: {dx[0]} {bedmachine.x.units}")

### Show the data

In [ ]:
fig, axs = plt.subplots(ncols=3, sharex=True, sharey=True, constrained_layout=True, dpi=300, figsize=(10, 2.5))
for ax, var, cmap in zip(axs, ["bed", "thickness", "mask"], [cmocean.cm.topo, None, None]):
    im = bedmachine[var].plot(ax=ax, cmap=cmap)
    ax.set_title(var)
    ax.set_aspect("equal")
    if var == "mask":
        im.colorbar.set_ticks(np.arange(4))
for ax in axs[1:]:
    ax.set_ylabel("")

## Show the alignment between the two datasets

In [ ]:
fig, ax = plt.subplots(figsize=(7, 4), dpi=300, constrained_layout=True)
for mask, name, ice_value, cmap in [
    (bedmachine.mask, "BedMachine", 3, "Reds"), (rtopo.amask, "RTopo", 2, "Blues")
]:
    im = mask.where((mask == 0) | (mask == ice_value)).plot(
        x="x", y="y", vmin=-2 * ice_value, vmax=2 * ice_value, cmap=cmap, alpha=0.5
    )
    im.colorbar.set_label(name)
    im.colorbar.set_ticks([-2 * ice_value, 0, ice_value])
    im.colorbar.set_ticklabels(["", "", ""] if name == "RTopo" else ["land", "ocean", "ice tongue"])
ax.set_title("Comparison of the masks of the two datasets")
ax.set_aspect("equal")

## Create the topography dataset

For the 1 km-resolution, we can keep the latitude grid of RTopo, since the meridional resolution of 1/120° is close to 1 km.
We reduce the longitudinal grid by combining every six grid cells into one.
This yields a resolution of 6/120° = 0.05°, which is also close to 1 km (shorter than 1 km in the North of the domain, longer than 1 km in the South).

In [ ]:
n_filter_lon = 6

In [ ]:
ds = xr.Dataset(
    {"grid_type": 2, "dlat": 1 / 120, "dlon": n_filter_lon / 120},
    attrs={
        "title": "Topography of the 79NG fjord for GETM",
        "author": "Markus Reinert (ORCID: 0000-0002-3761-8029)",
        "institution": "Leibniz Institute for Baltic Sea Research Warnemuende (IOW), Germany",
        "description": "Bathymetry and ice thickness of the 79NG fjord in Greenland "
                       "on a regular lat-lon grid with a resolution of approximately 1 km, "
                       "to be used with the regional ocean model GETM.",
        "comment": "When opening this dataset with ncview, use the option `-minmax all` "
                   "to see the full range of the topography.",
    },
)

# Add coordinates
ds.coords["lat"] = np.arange(79.1, 80.401, ds.dlat)
ds.coords["lon"] = np.arange(-23., -13.99, ds.dlon)

# Add attributes
ds.lat.attrs = {"long_name": "Latitude" , "units": "degree", "CRS": str(crs_latlon)}
ds.lon.attrs = {"long_name": "Longitude", "units": "degree", "CRS": str(crs_latlon)}
ds.dlat.attrs = {"long_name":   "Meridional resolution", "units": "degree"}
ds.dlon.attrs = {"long_name": "Longitudinal resolution", "units": "degree"}
ds.grid_type.attrs = {"description": "equi-distant spherical grid"}

# Check for consistency with RTopo
assert np.allclose(ds.lat, rtopo.lat), "Latitude does not match RTopo"
assert np.allclose(ds.lon, rtopo.lon[::n_filter_lon]), "Longitude does not match RTopo"

ds

### Check the grid resolution

In [ ]:
# Compute grid resolution in meter
LON, LAT = np.meshgrid(ds.lon, ds.lat)
dist_lat = gsw.distance(LON, LAT, axis=0)
dist_lon = gsw.distance(LON, LAT, axis=1)

assert np.allclose(ds.dlat, np.diff(ds.lat)), "meridional distance is not correct"
assert np.allclose(ds.dlon, np.diff(ds.lon)), "longitudinal distance is not correct"
assert np.allclose(dist_lat, dist_lat[0, 0]), "meridional distance in meter is not constant"
assert not np.allclose(dist_lon, dist_lon[0, 0]), "longitudinal distance in meter is almost constant"

print(f"{ds.dlat.long_name}:   {ds.dlat:.5f}° ({dist_lat[0, 0]:.2f} m)")
print(f"{ds.dlon.long_name}: {ds.dlon:.5f}° ({dist_lon.min():.2f} m to {dist_lon.max():.2f} m)")

### Add Cartesian coordinates

In [ ]:
X, Y = transform_latlon_to_xy(*xr.broadcast(ds.lat, ds.lon))
ds.coords["x"] = (["lat", "lon"], X)
ds.coords["y"] = (["lat", "lon"], Y)
ds.x.attrs = {"long_name": crs_cartesian.axis_info[0].name, "units": "m", "CRS": str(crs_cartesian)}
ds.y.attrs = {"long_name": crs_cartesian.axis_info[1].name, "units": "m", "CRS": str(crs_cartesian)}

### Reduce the longitudinal resolution of RTopo

In [ ]:
# Why does the selection work without method="nearest"?
ice_base = rtopo.ice_base_topography.rolling(lon=n_filter_lon, center=True).mean().sel(lon=ds.lon)
bed_rock = rtopo.bedrock_topography .rolling(lon=n_filter_lon, center=True).mean().sel(lon=ds.lon)

### Create the mask

#### Compute the ocean mask

We take as water/ocean points all grid cells where
 * the bedrock is below sea level **and**
 * the base of the ice is above the sea floor.

According to the check we made when loading RTopo, this fits with the RTopo mask.

We remove points where the ocean is only 2 m shallow or less, since these grid cells might fall dry during ebb.
Masking these points gives a clearly simpler domain boundary and reduces problems with misaligning ice/ocean data.

In [ ]:
ds["mask"] = (
    ("lat", "lon"),
    ((bed_rock < -2) & (ice_base > bed_rock)).data,
    {
        "long_name": "Ocean mask",
        "source": "RTopo-2.0.4 (Schaffer et al. 2019) with modifications",
        "description": "The ocean mask is True on all water points "
                       "and False on points with land or grounded ice.",
    },
)
ds.mask.plot(cbar_kwargs={"ticks": [0, 1]})
ds.mask

#### Remove an isolated lake

This lake has no connection with the ocean and is thus irrelevant for the model.
There is another isolated lake in the South, but this area will be cropped later

In [ ]:
mask_lake = ds.mask.sel(lat=slice(80.3, 80.32)).sel(lon=-16.35, method="nearest")
print(f"Lake contains {np.count_nonzero(mask_lake)} water points that are put to land now.")
mask_lake[...] = False
ds.mask.plot(cbar_kwargs={"ticks": [0, 1]});

#### Make the island in the South rectangular

This island is outside the domain of interest and within the buffer zone of the open boundary, so not physically relevant for the model.
Making it rectangular prevents numerical problems at this position.

In [ ]:
mask_point = ds.mask.sel(lat=79.175, lon=-17.7, method="nearest")
assert mask_point, "selected point is not an ocean point"
mask_point[...] = False
ds.mask.plot(cbar_kwargs={"ticks": [0, 1]});

#### Mask a bay with grounded ice

A bay in the northern part of the grounding line appears as ungrounded ice (thus ocean) in RTopo, even though the ice is probably grounded in this area.
In this bay, the water column thickness computed from RTopo is basically constant; it takes on exactly two values, which are both very close to 1.1 m.
Having a constant water column thickness over an extended area, despite varying ice and bedrock topography, is unrealistic and suggests interpolation issues in this area.
Even if the water column thickness of 1.1 m was realistic, it would rather not be important for the system.
Furthermore, also the ice thickness from BedMachine suggests that the ice in this bay is grounded, so we put this area to grounded ice.

Note that in the original RTopo data set, there are a few more points along the fjord walls at which the water column thickness is exactly one of these two values of about 1.1 m.

In [ ]:
mask_bay = ds.mask.sel(lat=slice(79.5, None), lon=slice(-22.35))
print(f"Bay contains {np.count_nonzero(mask_bay)} ocean points that are put to land now.")
mask_bay[...] = False
ds.mask.plot(cbar_kwargs={"ticks": [0, 1]});

#### Correct the mask in the Southwest

The masks of RTopo and BedMachine do not match in the southwestern corner of the 79NG fjord.
We put this part to grounded ice, as suggested by BedMachine, to improve numerical stability.

In [ ]:
mask_southwest = ds.mask.sel(lat=slice(None, 79.29), lon=slice(-22))
print(f"Southwestern corner of the fjord contains {np.count_nonzero(mask_southwest)} ocean points that are put to land now.")
mask_southwest[...] = False
ds.mask.plot(cbar_kwargs={"ticks": [0, 1]});

### Add the bathymetry

In [ ]:
ds["bathymetry"] = (
    ("lat", "lon"),
    -bed_rock.data.astype(np.float64),
    {
        "long_name": "Bottom depth",
        "units": "m",
        "positive": "down",
        "source": "RTopo-2.0.4 (Schaffer et al. 2019)",
    },
)
ds["bathymetry"] = ds.bathymetry.where(ds.mask)
ds.bathymetry.plot(cmap=cmocean.cm.deep)
plt.title("Bathymetry");

### Process the ice thickness

In [ ]:
# Interpolate the data to the model grid (using Cartesian coordinates)
# and apply the mask of the model grid
ds["glIceD"] = bedmachine.thickness.interp(x=ds.x, y=ds.y) * ds.mask

# Remove two isolated ice columns.
# The original data does not show isolated ice columns in these places,
# so they can be considered artificats due to the misalignment of the masks.
for lat, lon in [(79.8, -20.0), (80.17, -19.5)]:
    i_lat = abs(ds.lat - lat).argmin()
    i_lon = abs(ds.lon - lon).argmin()
    # Check that this is indeed an isolated ice column surrounded by non-ice points
    assert ds.glIceD[i_lat, i_lon] > 0, "selected point is not ice"
    for i in [-1, 0, 1]:
        for j in [-1, 0, 1]:
            if i != 0 or j != 0:
                assert ds.glIceD[i_lat + i, i_lon + j] == 0, "selected ice point is not isolated"
    print(
        f"Setting isolated ice column at {ds.lat.data[i_lat]:.2f}°N, {-ds.lon.data[i_lon]:.2f}°W",
        f"with thickness {ds.glIceD.data[i_lat, i_lon]:5.2f} m to 0.",
    )
    ds.glIceD[i_lat, i_lon] = 0

ds.glIceD.attrs = {
    "long_name": "Glacial ice thickness",
    "units": "m",
    "source": "BedMachine v5.5 (Morlighem et al. 2022)",
}

ds.glIceD.where(ds.glIceD).plot(cmap=cmocean.cm.ice)
plt.title("Ice thickness");

### Crop to the model domain

The chosen extent is 4 cells larger than the domain of interest, so that the open boundaries and the sponge zones (3 grid points adjacent to each open boundary) are *not* within the domain of interest.
Note that there is no open boundary in the West.

In [ ]:
lon_min = -23
lon_max = -15. + 4.5 * ds.dlon
lat_min = 79.2 - 4.5 * ds.dlat
lat_max = 80.3 + 4.5 * ds.dlat
ds = ds.sel(lon=slice(lon_min, lon_max), lat=slice(lat_min, lat_max))

print(f"Topography has {ds.bathymetry.size} grid points")
print(f"of which {np.count_nonzero(ds.mask) / ds.bathymetry.size * 100:2.0f} %",
      f"({np.count_nonzero(ds.mask):5}) are water")
print(f"     and {np.count_nonzero(~ds.mask) / ds.bathymetry.size * 100:2.0f} %",
      f"({np.count_nonzero(~ds.mask):5}) are land.")

ds

In [ ]:
lon_size = int(config.get_text("getm/domain/ih"))
lat_size = int(config.get_text("getm/domain/jh"))
assert lon_size == ds.lon.size, "grid size in longitude does not match configuration"
assert lat_size == ds.lat.size, "grid size in latitude does not match configuration"

### Smooth the topography

#### Analyse local slopes of the topography

In [ ]:
fig, axs = plt.subplots(2, 2, sharex=True, sharey=True, constrained_layout=True, figsize=(10, 7), dpi=300)
for axs_row, dim in zip(axs, ["lat", "lon"]):
    for ax, var in zip(axs_row, ["bathymetry", "glIceD"]):
        data = ds.glIceD.where(ds.glIceD) if var == "glIceD" else ds[var]
        data.diff(dim).plot(ax=ax)
        ax.set_title(f"Slope of {var} in {dim}-direction")

#### Apply a running mean

For a running mean of size 5×5, we pass the arguments `2, 2` to `smooth_2D_array`.

Note:
The ice is smoothed between ice and ocean, resulting in a larger ice tongue after the smoothing than before.
This might be beneficial to avoid large slopes at the calving front.
To only smooth the ice tongue over its extent, change `where(ds.mask)` to `where(ds.glIceD)`.

In [ ]:
bathy_smooth = smooth_2D_array(ds.bathymetry, 2, 2)
ice_smooth = smooth_2D_array(ds.glIceD.where(ds.mask), 2, 2)
ice_smooth = np.where(~np.isnan(ice_smooth), ice_smooth, 0)

#### Show differences before and after smoothing

In [ ]:
fig, axs = plt.subplots(ncols=2, sharex=True, sharey=True, constrained_layout=True, figsize=(10, 4), dpi=300)
(ds.bathymetry - bathy_smooth).plot(ax=axs[0])
(ds.glIceD - ice_smooth).plot(ax=axs[1]);

#### Analyse slopes of the smoothed data

In [ ]:
fig, axs = plt.subplots(2, 2, sharex=True, sharey=True, constrained_layout=True, figsize=(10, 7), dpi=300)
for axs_row, dim in zip(axs, [0, 1]):
    for ax, var in zip(axs_row, ["bathymetry", "glIceD"]):
        diff = np.diff(bathy_smooth if var == "bathymetry" else np.where(ice_smooth, ice_smooth, np.nan), axis=dim)
        vmax = max(np.nanmax(diff), -np.nanmin(diff))
        im = ax.pcolormesh(diff, vmax=vmax, vmin=-vmax, cmap=cmocean.cm.balance)
        fig.colorbar(im, ax=ax)
        ax.set_title(f"Slope of {var} in {dim}-direction")

#### Replace topography with smoothed data

In [ ]:
ds.bathymetry.data = bathy_smooth
ds.glIceD.data = ice_smooth

### Mask grounded ice

We determine if ice is floating or grounded using the reference density $\rho_0$ instead of the actual sea water density $\rho$, which is a function of space and time.
The difference between these two densities is generally less than 1 %.
See Section 2.2.1 of Reinert et al. (2023) for details.

The reference density is also used by GETM to initialize the ice in its (approximate) vertical equilibrium position.

We also mask out points where the ice is less than 2 m above the sea floor, which might be grounded during ebb tide.
This is analog to our computation of the (inital) mask above.

In [ ]:
rho_0 = float(config.get_text("getm/parameters/rho_0"))
rho_ice = float(config.get_text("getm/ice/rho_ice"))

print(f"The reference density is rho_0 = {rho_0:6.1f} kg/m³,")
print(f"glacial ice density is rho_ice = {rho_ice:6.1f} kg/m³ ({rho_ice/rho_0*100:.1f} % of rho_0).")

In [ ]:
is_grounded = ds.bathymetry < ds.glIceD * rho_ice / rho_0 + 2

is_grounded.where(ds.mask).plot(cmap="Reds")
plt.title("Points where the ice is grounded (dark red)")

print(f"There are {np.count_nonzero(is_grounded)} ocean points with grounded ice that are put to land now.")
ds["mask"] &= ~is_grounded
ds["bathymetry"] = ds.bathymetry.where(ds.mask)
ds["glIceD"] *= ds.mask

### Determine the grounding line

The grid points of the grounding line are the westernmost points of the water domain at 22.4°W and to the West of it.

In [ ]:
gline = (-1 * ds.mask).argmin("lon")
gline = gline.where(gline <= abs(ds.lon + 22.4).argmin(), other=-1, drop=True)

### Show the topography

#### Compute the coordinates of the grounding line

We show the grounding line in a step-wise manner along the western boundary of the grid cells that belong to the grounding line.

In [ ]:
gline_lat = []
gline_lon = []
for lon, lat in zip(ds.lon.isel(lon=gline), gline.lat):
    gline_lat += [lat - ds.dlat / 2, lat + ds.dlat / 2]
    gline_lon += [lon - ds.dlon / 2, lon - ds.dlon / 2]

#### Create the figure

In [ ]:
fig, axs = plt.subplots(ncols=3, sharex=True, sharey=True, constrained_layout=True, figsize=(15, 5), dpi=300)

ax = axs[0]
ds.bathymetry.plot(ax=ax, cmap=cmocean.cm.deep)
ax.set_title(f"Bathymetry around 79NG\nfrom {ds.bathymetry.source}")
ax.plot(gline_lon, gline_lat, "r", lw=1, solid_capstyle="butt", label="grounding line")
ax.legend(loc="lower left")

ax = axs[1]
ds.glIceD.where(ds.glIceD).plot(ax=ax, cmap=cmocean.cm.ice_r)
ax.set_title(f"Ice thickness of 79NG\nfrom {ds.glIceD.source}")
ax.set_ylabel("")

ax = axs[2]
im = (ds.bathymetry - ds.glIceD * rho_ice / rho_0).plot(ax=ax, cmap=cmocean.cm.amp)
im.colorbar.set_label("Water column thickness [m]")
ax.set_title(f"Water column thickness around 79NG\nusing $\\rho_0 = {rho_0:4.0f}$ kg/m³ and $\\rho_{{\\text{{ice}}}} = {rho_ice:4.0f}$ kg/m³")
ax.set_ylabel("")

fig.savefig("figures/topography_1km.png")

### Save the dataset

In [ ]:
filename = config.get_file_path("getm/rivers/river_info")
with open(filename, "w") as f:
    f.write("# Indices of the grid points along the 79NG grounding line\n")
    f.write("# on the GETM topography with 1 km resolution\n")
    f.write("# Subglacial runoff is uniformly distributed over these points.\n")
    f.write(f"{gline.size}\n")
    for ilon in gline:
        f.write(f"{ilon.data + 1} {np.where(ds.lat == ilon.lat)[0][0] + 1} runoff\n")
print(f"Saved the grounding line info as {filename!r}.")

In [ ]:
filename = config.get_file_path("getm/domain/bathymetry")
ds.to_netcdf(filename)
print(f"Saved the topography as {filename!r}.")